# Advent Of Code 2023

## Day 1: Trebuchet?!

In [ ]:
# Part 1
def extract_number(from_line: str) -> int:
    clean_line = ''.join(char for char in from_line if char.isdigit())
    return int(clean_line[0] + clean_line[-1])

with open('files/day1-input', 'r', encoding='utf-8') as input_file:
    print(f"The sum of all the calibration values is {sum(extract_number(line) for line in input_file)}")

In [ ]:
# Part 2
DIGIT_REPLACEMENT = {
    'one': 1,
    'two': 2,
    'three': 3,
    'four': 4,
    'five': 5,
    'six': 6,
    'seven': 7,
    'eight': 8,
    'nine': 9
}

def extract_number(from_line: str) -> int:
    
    first_digit, last_digit = None, None

    for (index, char) in enumerate(from_line):
        
        # Match regular digit
        if char.isdigit():
            last_digit = char
        
        else:
            # Match spelled out digit
            matches = [value for digit, value in DIGIT_REPLACEMENT.items() if from_line[index:].startswith(digit)]
            if any(matches):
                last_digit = str(matches[0])
    
        if first_digit is None:
            first_digit = last_digit
            
    return int(first_digit + last_digit)

with open('files/day1-input', 'r', encoding='utf-8') as input_file:
    print(f"The sum of all the calibration values is {sum(extract_number(line) for line in input_file)}")

## Day 2 - Cube conundrum

In [ ]:
# Part 1
import re

MAX_AMOUNTS = {
    "red":   12,
    "green": 13,
    "blue":  14,
}

RE_PARSE_GAME = re.compile(r"Game (\d+): (.*)")
RE_PARSE_DRAW = re.compile(r"(\d+) (red|green|blue)")

def process_game(line: str) -> tuple[int, int]:
    """Process the game result given as a string
    Returns a tuple containing the id of the game and True if the game is possible.
    """
    game_id, game_content = RE_PARSE_GAME.match(line).groups()

    return int(game_id), all(
        int(number) <= MAX_AMOUNTS[color]
        for number, color in (
            RE_PARSE_DRAW.search(bit).groups()
            for draw in game_content.split(sep=";")
            for bit in draw.split(sep=",")
        )
    )

with open('files/day2-input', 'r', encoding='utf-8') as input_file:
    print(f"The sum of all possible game IDs is {sum(id for id, result in (process_game(line) for line in input_file) if result)}")

In [ ]:
# Part 2
import re
import operator
from functools import reduce

RE_PARSE_GAME = re.compile(r"Game (\d+): (.*)")
RE_PARSE_DRAW = re.compile(r"(\d+) (red|green|blue)")

def process_game_power(line: str) -> int:
    """Process the game result given as a string
    Returns the power of the minimal possible set of cubes for the game.
    """
    game_id, game_content = RE_PARSE_GAME.match(line).groups()

    min_amounts = dict()
    for number, color in (RE_PARSE_DRAW.search(bit).groups() for draw in game_content.split(sep=";") for bit in draw.split(sep=",")):
        min_amounts[color] = max(int(number), min_amounts.get(color, 0))
        
    return reduce(operator.mul, min_amounts.values(), 1)

with open('files/day2-input', 'r', encoding='utf-8') as input_file:
    print(f"The sum of all possible game IDs is {sum(process_game_power(line) for line in input_file)}")

## Day 3 - Gear ratios

In [ ]:
# Part 1
import re

RE_FIND_NUMBER = re.compile(r"\d+")

with open('files/day3-input', encoding='utf-8') as input_file:
    engine: tuple[str, ...] = tuple(line.strip() for line in input_file)

# Compute boundaries
ENGINE_HEIGHT = len(engine)
ENGINE_WIDTH = len(engine[0])

total = 0

for row, line in enumerate(engine):
    for match in RE_FIND_NUMBER.finditer(line):
        if any(
            char != "." and not char.isdigit()
            for subline in engine[max(0, row - 1) : min(ENGINE_HEIGHT, row + 2)]
            for char in subline[max(0, match.start() - 1) : min(ENGINE_WIDTH, match.end() + 1)]
        ):
            total += int(match.group(0))

print(f"The sum of all part number is {total}")

In [ ]:
# Part 2
import re

RE_FIND_NUMBER = re.compile(r"\d+")

with open('files/day3-input', encoding='utf-8') as input_file:
    engine: tuple[str, ...] = tuple(line.strip() for line in input_file)

# Compute boundaries
ENGINE_HEIGHT = len(engine)
ENGINE_WIDTH = len(engine[0])

# This dictionary uses as key the position of a gear, and holds as value the list of part number adjacent to that gear
gears: dict[tuple[int, int], list[int]] = dict()

# Check every number for adjacent gear ; add it to gears dictionary
for row, line in enumerate(engine):
    for match in RE_FIND_NUMBER.finditer(line):
        for i in range(max(0, row - 1), min(ENGINE_HEIGHT, row + 2)):
            for j in range(max(0, match.start() - 1), min(ENGINE_WIDTH, match.end() + 1)):
                if engine[i][j] == "*":
                    gears[(i, j)] = gears.get((i, j), []) + [int(match.group(0))]


print(f"The sum of all part number is {sum(parts[0] * parts[1] for parts in gears.values() if len(parts) == 2)}")

## Day 4 - Scratchcards

In [ ]:
# Part 1
import re

RE_SCRATCHCARD = re.compile(r"Card\s+(\d+):\s*((?:\d+\s*)*) \| ((?:\s*\d+)*)\s*")

def compute_scratchcard(card: str) -> int:
    """Computes the score of a scratchcard given its string representation"""
    card_id, winning_numbers, your_numbers = RE_SCRATCHCARD.match(card).groups()
    return int(2**(sum(1 for number in your_numbers.split() if number and number in winning_numbers.split()) - 1))
    
with open('files/day4-input', encoding='utf-8') as input_file:
    print(f"The sum of all the scratchcards score is {sum(compute_scratchcard(line) for line in input_file)}")
    

In [ ]:
# Part 2
import re

RE_SCRATCHCARD = re.compile(r"Card\s+(\d+):\s*((?:\d+\s*)*) \| ((?:\s*\d+)*)\s*")

# This dictionary stores the number of each scratchcard (<card ID>: <number of cards>)
count_scratchcards: dict[int, int] = dict()

def compute_scratchcard(card: str) -> None:
    """Computes the scratchcards won by a scratchcard given its string representation"""
    card_id, winning_numbers, your_numbers = RE_SCRATCHCARD.match(card).groups()
    card_id = int(card_id)
    
    # Compute score
    score = sum(1 for number in your_numbers.split() if number and number in winning_numbers.split())
    
    # Update counts
    count_scratchcards[card_id] = count_scratchcards.get(card_id, 0) + 1
    for i in range(card_id + 1, card_id + score + 1):
        count_scratchcards[i] = count_scratchcards.get(i, 0) + count_scratchcards[card_id]
    
with open('files/day4-input', encoding='utf-8') as input_file:
    [compute_scratchcard(line) for line in input_file]
    print(f"The sum of all the scratchcards score is {sum(count_scratchcards.values())}")
    

## Day 5 - If You Give A Seed A Fertilizer

In [ ]:
# Part 1
import re

RE_READ_SEEDS = re.compile(r"\d+")
RE_READ_MAP = re.compile(r"(?P<destination>\d+) (?P<source>\d+) (?P<length>\d+)", flags=re.MULTILINE)

class TransformMap:
    
    def __init__(self, string_map):
        self.ranges = [
            (range(source, source + length), destination - source)
            for destination, source, length in map(lambda match: map(int, match), RE_READ_MAP.findall(string_map))
        ]
        
    def __getitem__(self, value):
        for source, offset in self.ranges:
            if value in source:
                return value + offset
        else:
            return value


with open('files/day5-input', encoding='utf-8') as input_file:
    seeds, *maps = input_file.read().split(sep="\n\n")

seeds = [int(seed) for seed in RE_READ_SEEDS.findall(seeds)]
maps = [TransformMap(map_str) for map_str in maps]

results = seeds[:]
for tmap in maps:
    results = [tmap[i] for i in results]

print(f"The lowest location number where a seed can be planted is {min(results)}")

In [ ]:
# Part 2
import re

RE_READ_SEEDS = re.compile(r"\d+")
RE_READ_MAP = re.compile(r"(?P<destination>\d+) (?P<source>\d+) (?P<length>\d+)", flags=re.MULTILINE)

class TransformMap:
    
    def __init__(self, string_map):
        self.ranges = [
            (range(source, source + length), destination - source)
            for destination, source, length in map(lambda match: map(int, match), RE_READ_MAP.findall(string_map))
        ]
        
    def applyToRange(self, r_from):
        
        ranges_from = [r_from]
        ranges_to = []
        
        for range_source, offset in self.ranges:
            for range_from in ranges_from[:]:
                intersection = range(max(range_from.start, range_source.start), min(range_from.stop, range_source.stop))
                if not intersection.start < intersection.stop:
                    continue
                    
                ranges_to.append(range(intersection.start + offset, intersection.stop + offset))
                
                # Replace the range_from by anything not in the intersection (on the left or on the right) in ranges_from
                ranges_from.remove(range_from)
                
                if range_from.start < intersection.start:
                    ranges_from.append(range(range_from.start, intersection.start))
                    
                if intersection.stop < range_from.stop:
                    ranges_from.append(range(intersection.stop, range_from.stop))
        
        # Return all ranges_to + what is left untouched in ranges_from
        return ranges_to + ranges_from

with open('files/day5-input', encoding='utf-8') as input_file:
    seeds, *maps = input_file.read().split(sep="\n\n")

seeds = [int(seed) for seed in RE_READ_SEEDS.findall(seeds)]
seeds = [range(start, start + length) for start, length in zip(seeds[::2], seeds[1::2])]

t_maps = [TransformMap(map_str) for map_str in maps]

results = seeds
for t_map in t_maps:
    results = [r for x in results for r in t_map.applyToRange(x)]
    
print(f"The lowest location number where a seed can be planted is {min(r.start for r in results)}")

## Day 6 - Wait For It

In [ ]:
# Part 1
import re
from functools import reduce
import operator as op
from typing import Callable

RE_NUMBER = re.compile(r"\d+")

def modelize_race(duration: int) -> Callable[[int], [int]]:
    """This function returns another function which modelize a race
    for a given duration. The returned function takes the time spent
    pressing the button and returns the distance reached
    """
    return lambda charge: charge * (duration - charge)

with open('files/day6-input', encoding='utf-8') as input_file:
    races = [
        (int(time), int(record))
        for time, record in zip(RE_NUMBER.findall(input_file.readline()), RE_NUMBER.findall(input_file.readline()))
    ]

nb_winning_ways = [
    sum(1 for duration in range(time) if modelize_race(time)(duration) > record)
    for time, record in races
]

print(f"Multiplying the number of ways to win each race yields {reduce(op.mul, nb_winning_ways, 1)}")

In [ ]:
# Part 2

with open('files/day6-input', encoding='utf-8') as input_file:
    time = int(RE_NUMBER.search(input_file.readline().replace(" ", "")).group())
    record = int(RE_NUMBER.search(input_file.readline().replace(" ", "")).group())
    
race = modelize_race(time)

print(f"The record is beatable in {sum(1 for duration in range(time) if race(duration) > record)} ways")

## Day 7 - Camel Cards

In [ ]:
# Part 1
from collections import Counter
from enum import Enum

FACE_CARDS = {
    'A': 14,
    'K': 13,
    'Q': 12,
    'J': 11,
    'T': 10,
}

class CamelHand:
    """This class represents a hand of Camel Cards.
    The index of the hand is a base-14 integer with 6 (base-14) digits.
    The most significant digit represent the hand type, while every
    other digit in order of significance represent the cards.
    This index makes sorting the hands by value trivial.
    """
    
    class HandTypes(Enum):
        FIVE_OF_A_KIND  = 6
        FOUR_OF_A_KIND  = 5
        FULL_HOUSE      = 4
        THREE_OF_A_KIND = 3
        TWO_PAIRS       = 2
        ONE_PAIR        = 1
        HIGH_CARD       = 0
    
    def __init__(self, hand):
        self.hand = [FACE_CARDS[card] if card in FACE_CARDS else int(card) for card in hand]
        self.index = sum(x * 14**n for n, x in enumerate(self.hand[::-1])) + self.get_hand_type().value * 14**5
    
    def get_hand_type(self):
        _, counts = zip(*Counter(self.hand).most_common())
        
        match counts[0]:
            case 5:
                return CamelHand.HandTypes.FIVE_OF_A_KIND
            case 4:
                return CamelHand.HandTypes.FOUR_OF_A_KIND
            case 3:
                return CamelHand.HandTypes.FULL_HOUSE if counts[1] == 2 else CamelHand.HandTypes.THREE_OF_A_KIND
            case 2:
                return CamelHand.HandTypes.TWO_PAIRS if counts[1] == 2 else CamelHand.HandTypes.ONE_PAIR
            case _:
                return CamelHand.HandTypes.HIGH_CARD
        

with open('files/day7-input', encoding='utf-8') as input_file:
    games = [(CamelHand(hand), int(bet)) for (hand, bet) in (line.split() for line in input_file)]
    
print(f"The total winnings are {sum(bet * (i + 1) for i, (hand, bet) in enumerate(sorted(games, key=lambda game: game[0].index)))}")

In [ ]:
# Part 2
from collections import Counter
from enum import Enum

FACE_CARDS = {
    'A': 14,
    'K': 13,
    'Q': 12,
    'T': 10,
    'J': 1,
}

class CamelHand:
    """This class represents a hand of Camel Cards.
    The index of a hand is a base-14 integer with 6 (base-14) digits.
    The most significant digit represent the hand type, while every
    other digit in descending order of significance represent the cards.
    This index makes sorting the hands by value trivial.
    """
    
    class HandTypes(Enum):
        FIVE_OF_A_KIND  = 6
        FOUR_OF_A_KIND  = 5
        FULL_HOUSE      = 4
        THREE_OF_A_KIND = 3
        TWO_PAIRS       = 2
        ONE_PAIR        = 1
        HIGH_CARD       = 0
    
    def __init__(self, hand):
        self.hand = [FACE_CARDS[card] if card in FACE_CARDS else int(card) for card in hand]
        self.index = sum(x * 14**n for n, x in enumerate((*self.hand[::-1], self.get_hand_type().value)))
        # if 'J' in hand:
        #     print(f"{hand} is {self.get_hand_type()}")
    
    def get_hand_type(self):
        counter = Counter(self.hand) # We don't count jokers here because they do not form hands by themselves
                
        if counter[FACE_CARDS['J']] == 5:
            # Edge case: only jokers in hand is a Five-of-a-king
            return CamelHand.HandTypes.FIVE_OF_A_KIND
        
        _, counts = zip(*((card, count) for (card, count) in counter.most_common() if card != FACE_CARDS['J']))
        
        match counts[0] + counter[FACE_CARDS['J']]:
            case 5:
                return CamelHand.HandTypes.FIVE_OF_A_KIND
            case 4:
                return CamelHand.HandTypes.FOUR_OF_A_KIND
            case 3:
                return CamelHand.HandTypes.FULL_HOUSE if len(counts) > 1 and counts[1] == 2 else CamelHand.HandTypes.THREE_OF_A_KIND
            case 2:
                return CamelHand.HandTypes.TWO_PAIRS if len(counts) > 1 and counts[1] == 2 else CamelHand.HandTypes.ONE_PAIR
            case _:
                return CamelHand.HandTypes.HIGH_CARD
        

with open('files/day7-input', encoding='utf-8') as input_file:
    games = [(CamelHand(hand), int(bet)) for (hand, bet) in (line.split() for line in input_file)]
    
print(f"The total winnings are {sum(bet * (i + 1) for i, (hand, bet) in enumerate(sorted(games, key=lambda game: game[0].index)))}")

## Day 8 - Haunted Wasteland

In [ ]:
# Part 1
import re
from itertools import cycle

RE_NODE = re.compile(r"(?P<Node>\w{3}) = \((?P<L>\w{3}), (?P<R>\w{3})\)")

START_NODE = "AAA"
END_NODE = "ZZZ"

with open('files/day8-input', encoding='utf-8') as input_file:
    _1, _2 = input_file.read().split(sep="\n\n")
    instructions = cycle(_1)
    map_network = {node['Node']: node for node in (RE_NODE.match(line).groupdict() for line in _2.split(sep="\n"))}

current_node = START_NODE
nb_steps = 0

while current_node != END_NODE:
    current_node = map_network[current_node][next(instructions)]
    nb_steps += 1

print(f"{nb_steps} step{'s' if nb_steps > 1 else ''} are required to reach {END_NODE}")

In [ ]:
# Part 2
import re
from itertools import cycle
from functools import reduce
import math

RE_NODE = re.compile(r"(?P<Node>\w{3}) = \((?P<L>\w{3}), (?P<R>\w{3})\)")

with open('files/day8-input', encoding='utf-8') as input_file:
    instructions, map_str = input_file.read().split(sep="\n\n")
    map_network = {node['Node']: node for node in (RE_NODE.match(line).groupdict() for line in map_str.split(sep="\n"))}

start_nodes = [node for node in map_network if node[-1] == 'A']
end_nodes = [node for node in map_network if node[-1] == 'Z']

def run(start_node):
    current_node = start_node
    nb_steps = 0
    
    for move in cycle(instructions):
        if current_node in end_nodes:
            return nb_steps
        
        current_node = map_network[current_node][move]
        nb_steps += 1


print(f"It takes {reduce(math.lcm, (run(node) for node in start_nodes), 1)} steps before we're only on nodes that end with a Z")

## Day 9 - Mirage Maintenance

In [ ]:
# Part 1
from functools import reduce

def extrapolate_sequence(sequence):
    differences = reduce(lambda acc, it: acc + [it[1] - it[0]], zip(sequence, sequence[1:]), [])

    if any(differences):
        differences.append(extrapolate_sequence(differences))
    else:
        differences.append(0)

    return sequence[-1] + differences[-1]

with open('files/day9-input', encoding='utf-8') as input_file:
    print(f"The sum of the extrapolated values is {sum(extrapolate_sequence(seq) for seq in (list(map(int, line.split())) for line in input_file))}")

In [ ]:
# Part 2
from functools import reduce

def extrapolate_sequence(sequence):
    differences = reduce(lambda acc, it: acc + [it[1] - it[0]], zip(sequence, sequence[1:]), [])

    if any(differences):
        differences.insert(0, extrapolate_sequence(differences))
    else:
        differences.insert(0, 0)

    return sequence[0] - differences[0]

with open('files/day9-input', encoding='utf-8') as input_file:
    print(f"The sum of the extrapolated values is {sum(extrapolate_sequence(seq) for seq in (list(map(int, line.split())) for line in input_file))}")

## Day 10 - Pipe Maze

In [ ]:
# Part 1
from operator import itemgetter

with open('files/day10-input', encoding='utf-8') as input_file:
    grid = [list(line.strip()) for line in input_file]

def get_tile_direction(i, j):
    
    match grid[i][j]:
        case '|':
            return [(i - 1, j, ['|', 'F', '7']), (i + 1, j, ['|', 'L', 'J'])]
        case '-':
            return [(i, j - 1, ['-', 'F', 'L']), (i, j + 1, ['-', '7', 'J'])]
        case 'L':
            return [(i - 1, j, ['|', 'F', '7']), (i, j + 1, ['-', '7', 'J'])]
        case 'J':
            return [(i - 1, j, ['|', 'F', '7']), (i, j - 1, ['-', 'F', 'L'])]
        case '7':
            return [(i + 1, j, ['|', 'L', 'J']), (i, j - 1, ['-', 'F', 'L'])]
        case 'F':
            return [(i + 1, j, ['|', 'L', 'J']), (i, j + 1, ['-', '7', 'J'])]
        case 'S':
            return [
                (i - 1, j, ['|', 'F', '7']), (i + 1, j, ['|', 'L', 'J']),
                (i, j - 1, ['-', 'F', 'L']), (i, j + 1, ['-', '7', 'J'])
            ]
        case _:
            return []

seen = [(i, j) for (i, row) in enumerate(grid) for (j, tile) in enumerate(row) if tile == 'S']
loop = [(*seen[0], 0)]

for tile in loop:
    i, j, distance = tile

    for (ii, jj, allowed_connections) in get_tile_direction(i, j):
        if (ii, jj) not in seen and ii in range(len(grid)) and jj in range(len(grid[ii])) and grid[ii][jj] in allowed_connections:
            seen.append((ii, jj))
            loop.append((ii, jj, distance + 1))


print(f"It takes {sorted(loop, key=itemgetter(2), reverse=True)[0][2]} steps along the loop from the starting position to the point farthest from the starting position")    

In [ ]:
# Part 2

with open('files/day10-input', encoding='utf-8') as input_file:
    grid = [list(line.strip()) for line in input_file]

def get_tile_direction(i, j):
    
    match grid[i][j]:
        case '|':
            return [(i - 1, j, ['|', 'F', '7']), (i + 1, j, ['|', 'L', 'J'])]
        case '-':
            return [(i, j - 1, ['-', 'F', 'L']), (i, j + 1, ['-', '7', 'J'])]
        case 'L':
            return [(i - 1, j, ['|', 'F', '7']), (i, j + 1, ['-', '7', 'J'])]
        case 'J':
            return [(i - 1, j, ['|', 'F', '7']), (i, j - 1, ['-', 'F', 'L'])]
        case '7':
            return [(i + 1, j, ['|', 'L', 'J']), (i, j - 1, ['-', 'F', 'L'])]
        case 'F':
            return [(i + 1, j, ['|', 'L', 'J']), (i, j + 1, ['-', '7', 'J'])]
        case 'S':
            return [
                (i - 1, j, ['|', 'F', '7']), (i + 1, j, ['|', 'L', 'J']),
                (i, j - 1, ['-', 'F', 'L']), (i, j + 1, ['-', '7', 'J'])
            ]
        case _:
            return []

loop = [(i, j) for (i, row) in enumerate(grid) for (j, tile) in enumerate(row) if tile == 'S']

for tile in loop:
    i, j = tile

    for (ii, jj, allowed_connections) in get_tile_direction(i, j):
        if (ii, jj) not in loop and ii in range(len(grid)) and jj in range(len(grid[ii])) and grid[ii][jj] in allowed_connections:
            loop.append((ii, jj))

def count_enclosed_cells_in_row(row_i):
    
    is_inside = False
    start_of_edge = None
    count = 0

    for (j, tile) in enumerate(grid[row_i]):
        if (row_i, j) in loop:
            if tile == '|':
                is_inside = not is_inside
            elif tile in ('F', 'L'):
                start_of_edge = tile
            elif tile in ('7', 'J'):
                if (start_of_edge, tile) in (('F', 'J'), ('L', '7')):
                    is_inside = not is_inside
                start_of_edge = None
        elif is_inside:
            count += 1

    return count

print(f"{sum(count_enclosed_cells_in_row(i) for i in range(len(grid)))} tiles are enclosed in the loop")

## Day 11 - Cosmic Expansion

In [ ]:
# Part 1
import itertools

with open('files/day11-input', encoding='utf-8') as input_file:
    universe = [list(line.strip()) for line in input_file]
    universe_t = list(map(list, zip(*universe)))

EXPANDED_ROWS = [i for i, row in enumerate(universe) if all(space == '.' for space in row)]
EXPANDED_COLS = [i for i, col in enumerate(universe_t) if all(space == '.' for space in col)]

galaxies = {(i, j) for i, row in enumerate(universe) for j, space in enumerate(row) if space == "#"}

print(f"""The sum of the distances between each pair of galaxies is {sum(
    abs(ax - bx) + sum(1 for i in EXPANDED_ROWS if ax < i < bx or bx < i < ax) +
    abs(ay - by) + sum(1 for j in EXPANDED_COLS if ay < j < by or by < j < ay)
    for (ax, ay), (bx, by) in itertools.combinations(galaxies, 2)
)}""")

In [ ]:
# Part 2
import itertools

with open('files/day11-input', encoding='utf-8') as input_file:
    universe = [list(line.strip()) for line in input_file]
    universe_t = list(map(list, zip(*universe)))

EXPANDED_ROWS = [i for i, row in enumerate(universe) if all(space == '.' for space in row)]
EXPANDED_COLS = [i for i, col in enumerate(universe_t) if all(space == '.' for space in col)]

EXPANSION_RATE = 999_999

galaxies = {(i, j) for i, row in enumerate(universe) for j, space in enumerate(row) if space == "#"}

print(f"""The sum of the distances between each pair of galaxies is {sum(
    abs(ax - bx) + sum(EXPANSION_RATE for i in EXPANDED_ROWS if ax < i < bx or bx < i < ax) +
    abs(ay - by) + sum(EXPANSION_RATE for j in EXPANDED_COLS if ay < j < by or by < j < ay)
    for (ax, ay), (bx, by) in itertools.combinations(galaxies, 2)
)}""")

## Day 12 - Hot Springs

In [ ]:
# Part 1
import re
from itertools import combinations

RE_PARSE_RECORD = re.compile(r"([\.\#\?]+) ([\d,]+)")

with open('files/day12-input', encoding='utf-8') as input_file:
    records = [
        (record, tuple(map(int, group.split(sep=','))))
        for record, group in (RE_PARSE_RECORD.match(line).groups() for line in input_file)
    ]

count = 0

for record, groups in records:
    
    # We build a regexp allowing to check if a complete line matches the incomplete record
    re_record = re.compile("".join("." if c == "?" else '\\' + c for c in record))
    
    # We build every possible combination of groups in the line size
    arrangements_len = len(record) - sum(groups) + 1
    for arrangement in combinations(range(arrangements_len), len(groups)):
        blanks = [b - a for a, b in zip((0, *arrangement), arrangement)]
        
        line = "".join(b * "." + a * "#" for a, b in zip(groups, blanks)).ljust(len(record), '.')
        # print(line)
        
        # If the built line matches the regexp built previously, we increase the count
        if re_record.match(line):
            count += 1

print(count)

In [ ]:
# Part 2
import re
from itertools import combinations
from functools import cache

RE_PARSE_RECORD = re.compile(r"([\.\#\?]+) ([\d,]+)")

@cache
def count_solutions(record: str, groups: list[int]) -> int:
    # Use functools.cache to completely obliterate the complexity of the problem :D
        
    if not record:
        return 1 if not groups else 0
    
    if not groups:
        return 0 if '#' in record else 1
    
    if record[0] == '.':
        return count_solutions(record[1:], groups)
    
    if record[0] == '?':
        return count_solutions(record[1:], groups) + count_solutions('#' + record[1:], groups)
    
    if record[0] == '#':
        if len(record) >= groups[0] and all(c != '.' for c in record[:groups[0]]) and (len(record) == groups[0] or record[groups[0]] != '#'):
            return count_solutions(record[groups[0] + 1:], groups[1:])
        else:
            return 0 # No solution possible

with open('files/day12-input', encoding='utf-8') as input_file:
    records = [
        ('?'.join(record for _ in range(5)), tuple(map(int, ','.join(group for _ in range(5)).split(sep=','))))
        for record, group in (RE_PARSE_RECORD.match(line).groups() for line in input_file)
    ]

# records = records[:1]

print(sum(count_solutions(record, groups) for record, groups in records))

## Day 13 - Point of Incidence

In [ ]:
# Part 1

def find_mirror(pattern: list[str]) -> int:
    for i in range(len(pattern) - 1):
        if all(x1 == x2 for (x1, x2) in zip(pattern[i::-1], pattern[i+1:])):
            return i + 1
    else:
        return 0

with open('files/day13-input') as input_file:
    patterns = [pattern.split(sep="\n") for pattern in input_file.read().split(sep="\n\n")]
    
print(sum(100 * find_mirror(pattern) + find_mirror(["".join(line) for line in zip(*pattern)]) for pattern in patterns))

In [ ]:
# Part 2

def find_mirror(pattern: list[str]) -> int:
    for i in range(len(pattern) - 1):
        # Count differences
        differences = 0
        for (x1, x2) in zip(pattern[i::-1], pattern[i+1:]):
            differences += sum(1 for c1, c2 in zip(x1, x2) if c1 != c2)
            
        if differences == 1:
            return i + 1
    else:
        return 0

with open('files/day13-input') as input_file:
    patterns = [pattern.split(sep="\n") for pattern in input_file.read().split(sep="\n\n")]
    
print(sum(100 * find_mirror(pattern) + find_mirror(["".join(line) for line in zip(*pattern)]) for pattern in patterns))

## Day 14 - Parabolic Reflector Dish

In [ ]:
# Part 1
from enum import Enum
from typing import Type

class TiltDirection(Enum):
    NORTH = (True, True)
    SOUTH = (True, False)
    WEST = (False, True)
    EAST = (False, False)

    def __new__(cls, transpose, reverse):
        obj = object.__new__(cls)
        obj.transpose = transpose
        obj.reverse = reverse
        return obj

def tilt(platform: list[str], direction: Type[TiltDirection]) -> None:
    """ The strategie for tilting the platform is the following
    - Transpose the matrix to
    - For each row of the platform (in the direction of the tilting):
    - Split the row on cube blocks (#)
    - Sort each subtring according to the direction
    - Join the sorted segment back into the original row
    - Join the rows into the original platform

    The operation is NOT done in-place
    """
    if direction.transpose: # Transpose
        platform = ["".join(line) for line in zip(*platform)]
    
    platform = [
        '#'.join(
            ''.join(sorted(segment, reverse=direction.reverse))
            for segment in line.split(sep='#')
        )
        for line in platform
    ]

    if direction.transpose: # Transpose back
        platform = ["".join(line) for line in zip(*platform)]

    return platform

def calculate_load(platform):
    """ Computes the total load on the north beam of the platform
    """
    return sum(
        sum(
            len(platform) - position if cell == 'O' else 0
            for cell in line
        )
        for position, line in enumerate(platform)
    )

with open('files/day14-input', encoding='utf-8') as input_file:
    platform = [line.strip() for line in input_file]

# Tilt the platform
platform = tilt(platform, TiltDirection.NORTH)

print(f"The total load on the north beam of the platform is {calculate_load(platform)}")

In [ ]:
# Part 2
# A simple brute-force sparkled with a little cycle detection
from itertools import cycle

with open('files/day14-input', encoding='utf-8') as input_file:
    platform = [line.strip() for line in input_file]

NB_CYCLES = 1_000_000_000
CYCLE_DIRECTIONS = (
    TiltDirection.NORTH,
    TiltDirection.WEST,
    TiltDirection.SOUTH,
    TiltDirection.EAST
)

hist = {}

for i in range(NB_CYCLES):

    if platform in hist.values(): # Cycle detected!
        cycle_start = list(hist.keys())[list(hist.values()).index(platform)]
        cycle_end = i
        
        platform = hist[cycle_start + (NB_CYCLES - cycle_start) % (cycle_end - cycle_start)]
        break
    
    else:
        hist[i] = platform

    for direction in CYCLE_DIRECTIONS:
        platform = tilt(platform, direction)

print(f"The total load on the north beam of the platform after {NB_CYCLES} cycles is {calculate_load(platform)}")

## Day 15

In [ ]:
# Part 1
from functools import reduce

def hash(line: str) -> int:
    return reduce(lambda value, char: (value + ord(char)) * 17 % 256, [*line], 0)

with open('files/day15-input', encoding='utf-8') as input_file:
    init_sequence = input_file.read().split(',')

print(f"The sum of the HASH algorithm on each instruction is {sum(hash(instruction) for instruction in init_sequence)}")

In [ ]:
# Part 2
import re

RE_PARSE_INSTRUCTION = re.compile(r"([a-z]*)([-=])(\d)?")

boxes = [{} for _ in range(256)]

def operate(instruction: str) -> None:
    label, operation, operand = RE_PARSE_INSTRUCTION.match(instruction).groups()

    box = boxes[hash(label)]

    match operation:
        
        case '=':
            box[label] = int(operand)

        case '-':
            if label in box:
                del box[label]

def compute_focusing_power() -> int:
    return sum(
        (box_i + 1) * (slot + 1) * focal_length
        for box_i, box in enumerate(boxes)
        for slot, (label, focal_length) in enumerate(box.items())
    )

with open('files/day15-input', encoding='utf-8') as input_file:
    init_sequence = input_file.read().split(',')

# Run the initialization sequence
for instr in init_sequence:
    operate(instr)

print(f"The focusing power of the lens configuration is {compute_focusing_power()}")

## Day 16 - The Floor Will Be Lava

In [5]:
# Part 1
from dataclasses import dataclass
from typing import Type

with open('files/day16-input', encoding='utf-8') as input_file:
    tiles = [[*line.strip()] for line in input_file]

@dataclass
class Beam:
    position: tuple[int, int]
    direction: tuple[int, int]

    def update(self):

        match tiles[position.x

visited_tiles: set[tuple[int, int]] = set()

beams: list[Type[Beam]] = [Beam(position=(0, 0), direction=(0, 1))]

while beams

display(tiles)

[['.', '|', '.', '.', '.', '\\', '.', '.', '.', '.'],
 ['|', '.', '-', '.', '\\', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '|', '-', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '|', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '\\'],
 ['.', '.', '.', '.', '/', '.', '\\', '\\', '.', '.'],
 ['.', '-', '.', '-', '/', '.', '.', '|', '.', '.'],
 ['.', '|', '.', '.', '.', '.', '-', '|', '.', '\\'],
 ['.', '.', '/', '/', '.', '|', '.', '.', '.', '.']]